In [1]:
import os
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='adamacs', ("Please move to the main directory")
from adamacs.pipeline import subject, session, equipment, surgery, event, trial, imaging, scan, model
from adamacs.ingest import session as isess
from adamacs.helpers import stack_helpers as sh
from adamacs.ingest import behavior as ibe
import datajoint as dj
from rspace_client.eln import eln
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
dj.__version__

[2023-05-12 21:44:55,745][INFO]: Connecting tobiasr@172.26.128.53:3306
[2023-05-12 21:44:55,793][INFO]: Connected tobiasr@172.26.128.53:3306


'0.14.0'

1. Get latest dataset from Natasha from database
2. load 2p movie to array. 
3. load top-video to array
4. find the Start and end frames of the movie with respect to the tracking gate 

In [2]:
import bisect
import numpy as np
from skimage.transform import resize
from tqdm import tqdm
import concurrent.futures
import cv2

def get_closest_timestamps(series, target_timestamp):
    # List to store the indices
    indices = []

    # For each timestamp in series1, find the closest timestamp in series2 and get its index
    for t1 in series:
        closest_index = closest_timestamp(target_timestamp, t1)
        indices.append(closest_index)
    return indices

# Function to find closest timestamp
def closest_timestamp(series, target_timestamp):
    index = bisect.bisect_left(series, target_timestamp)
    if index == 0:
        return 0
    if index == len(series):
        return len(series)-1
    before = series[index - 1]
    after = series[index]
    if after - target_timestamp < target_timestamp - before:
       return index
    else:
       return index-1


def selected_frames(video, frame_indices):
    """
    Resize selected frames of a video.

    Parameters:
    video (np.array): Input video as a numpy array of shape (frames, height, width, channels).
    frame_indices (np.array): Indices of the frames to resize and keep.
    new_size (tuple): The target height and width as a tuple (height, width).

    Returns:
    np.array: A numpy array of the resized video.
    """

    # Create a boolean mask of the same size as the number of frames
    mask = np.zeros(video.shape[0], dtype=bool)

    # Set the indices you're interested in to True
    mask[frame_indices] = True

    # Apply the mask to the video to get the frames you're interested in
    selected_frames = video[mask]



    return selected_frames



def resize_movie(movie, new_height, new_width):
    # Get the number of frames and color channels
    num_frames, _, _, num_channels = movie.shape
    
    # Initialize an empty array for the scaled movie
    scaled_movie = np.empty((num_frames, new_height, new_width, num_channels))
    
    # Iterate through each frame
    for i in tqdm(range(num_frames), desc="Resizing frames"):
        # Resize the frame and store it in the new array
        scaled_movie[i] = cv2.resize(movie[i], (new_width, new_height), interpolation = cv2.INTER_AREA)
    
    # Return the scaled movie
    return scaled_movie


def resize_frame(frame, new_height, new_width):
    return cv2.resize(frame, (new_width, new_height), interpolation = cv2.INTER_AREA)

def resize_movie_mt(movie, new_height, new_width):
    num_frames, _, _, num_channels = movie.shape
    scaled_movie = np.empty((num_frames, new_height, new_width, num_channels))

    with concurrent.futures.ThreadPoolExecutor() as executor:
        for i, resized_frame in tqdm(enumerate(executor.map(resize_frame, movie, [new_height]*num_frames, [new_width]*num_frames)), total=num_frames, desc="Resizing frames"):
            scaled_movie[i] = resized_frame

    return scaled_movie


import numpy as np
from skimage.transform import resize
import concurrent.futures



In [3]:
# defining queries

user = "NK"
userquery = (session.SessionUser * subject.User() & f'initials = "{user}"')
# get the latest session
usesession = session.Session.fetch(order_by='session_datetime DESC', as_dict=True)[0]["session_id"]
sessionquery = session.Session &  f'session_id = "{usesession}"'

scansi = (scan.Scan & sessionquery).fetch('scan_id')[0]
scanquery = scan.Scan & f'scan_id = "{scansi}"'


In [4]:
from pathlib import Path
import skvideo.io

# get the main data directory of the latest session of the specified user
# dir = (session.SessionDirectory & sessionquery).fetch('session_dir')
dir = (scan.ScanPath & scanquery).fetch('path')
directory = Path(dir[0])
pattern = '*top_video*.mp4'
files = list(directory.glob(pattern))
videodata = np.asarray(skvideo.io.vread(str(files[0])), dtype=np.uint8)
# videodata = np.asarray(videodata, dtype=np.uint8)


In [5]:
# get the blinking movie data of the latest session of the specified user
directory = Path(dir[0] + "/suite2p/plane0/reg_tif")
pattern = '*80_frame*.mp4'
files = list(directory.glob(pattern))
blinkvideodata = np.asarray(skvideo.io.vread(str(files[0])), dtype=np.uint8)
# blinkvideodata = np.asarray(blinkvideodata, dtype=np.uint8)

In [6]:
print(videodata.shape)
print(blinkvideodata.shape)

(36157, 1000, 1000, 3)
(9230, 256, 256, 3)


In [7]:
sh.display_volume_z(videodata,1)

interactive(children=(IntSlider(value=0, description='z', max=36156), Output()), _dom_classes=('widget-interac…

In [8]:
sh.display_volume_z(blinkvideodata,1)

interactive(children=(IntSlider(value=0, description='z', max=9229), Output()), _dom_classes=('widget-interact…

In [9]:
auxgatetimestamp_end = (event.Event()  &  "event_type='main_track_gate'" &  scanquery ).fetch('event_end_time')
auxgatetimestamp_start = (event.Event()  &  "event_type='main_track_gate'" &  scanquery ).fetch('event_start_time')
cameratimestamps = (event.Event()  &  "event_type='aux_cam'" & f"event_start_time>{auxgatetimestamp_start[0]}" & f"event_start_time<{auxgatetimestamp_end[0]}" & scanquery).fetch('event_start_time')
twoptimestamps = (event.Event()  &  "event_type='mini2p_frames'" &  scanquery ).fetch('event_start_time')
cameratimestamps = cameratimestamps - twoptimestamps[0]



In [20]:
aligned_cameraframes = get_closest_timestamps(twoptimestamps,cameratimestamps)

In [26]:
np.shape(aligned_cameraframes)

(9230,)

In [ ]:
np.unique(aligned_cameraframes)

In [32]:
np.shape(k|{0})

()

In [19]:
np.shape(cameratimestamps)

(36037,)

In [22]:

resliced_video = selected_frames(videodata, aligned_cameraframes)
resliced_video = np.asarray(resliced_video, dtype=np.uint8)

In [33]:
np.unique(resliced_video)

: 

In [ ]:
sh.display_volume_z(resliced_video,1)

In [15]:
# rescaled_movie = resize_movie_mt(blinkvideodata, np.shape(videodata)[1], np.shape(videodata)[2])
rescaled_movie = resize_movie_mt(blinkvideodata, 1000,1000)


Resizing frames: 100%|██████████| 9230/9230 [02:12<00:00, 69.72it/s] 


In [25]:
sh.display_volume_z(rescaled_movie,1)

interactive(children=(IntSlider(value=0, description='z', max=9229), Output()), _dom_classes=('widget-interact…

In [17]:
np.shape(rescaled_movie)

(9230, 1000, 1000, 3)

In [ ]:
frame_rate_video1

In [ ]:
timestamps_video1[0] - timestamps_video2[0]

In [ ]:
model.VideoRecording()